# Senario 1

MLflow setup:
No Tracking server
localfile store
localfile for artifact

In [2]:
import mlflow

In [3]:
print(f"tracking uri {mlflow.get_tracking_uri()}")

tracking uri file:///home/akogo/Desktop/MLOps/002-experinment-tracking/002-training/experinment-Tracking/running-mlflow-experinment/mlruns


In [ ]:
# listing all experiments
mlflow.search_experiments()

[<Experiment: artifact_location='file:///home/akogo/Desktop/MLOps/002-experinment-tracking/002-training/experinment-Tracking/running-mlflow-experinment/mlruns/0', creation_time=1730971541656, experiment_id='0', last_update_time=1730971541656, lifecycle_stage='active', name='Default', tags={}>]

## creating an experiment and logging new rub

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/11/07 09:27:42 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2024/11/07 09:27:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 'file:///home/akogo/Desktop/MLOps/002-experinment-tracking/002-training/experinment-Tracking/running-mlflow-experinment/mlruns/199277987339299424/db9c5b86083c46fd91dc88a6a78a3c9c/artifacts'


In [8]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///home/akogo/Desktop/MLOps/002-experinment-tracking/002-training/experinment-Tracking/running-mlflow-experinment/mlruns/199277987339299424', creation_time=1730971662724, experiment_id='199277987339299424', last_update_time=1730971662724, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///home/akogo/Desktop/MLOps/002-experinment-tracking/002-training/experinment-Tracking/running-mlflow-experinment/mlruns/0', creation_time=1730971541656, experiment_id='0', last_update_time=1730971541656, lifecycle_stage='active', name='Default', tags={}>]

## Interacting with the model registry

In [9]:
from mlflow.tracking import MlflowClient


client = MlflowClient()

In [11]:


from mlflow.exceptions import MlflowException

try:
    client.search_registered_models()
except MlflowException:
    print("It's not possible to access the model registry :(")



# Senario -2
cross functional team with one DS working on the model

## Mlfow setup

local tracking server
backend store sqlitedb
artifact store local server

mlflow server --backend-store-uri sqlite:///backend.db

## <MLlow Setup>

In [12]:
import mlflow

In [13]:
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

In [14]:
print(f"tracking URI{mlflow.get_tracking_uri()}")

tracking URIhttp://127.0.0.1:5000/


In [15]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1730972599645, experiment_id='0', last_update_time=1730972599645, lifecycle_stage='active', name='Default', tags={}>]

## creating experiment

In [16]:


from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")



2024/11/07 09:46:44 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2024/11/07 09:46:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/07 09:46:46 INFO mlflow.tracking._tracking_service.client: 🏃 View run popular-cod-605 at: http://127.0.0.1:5000/#/experiments/1/runs/5e8639781735427ba15d599d98a29a80.
2024/11/07 09:46:46 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1.


default artifacts URI: 'mlflow-artifacts:/1/5e8639781735427ba15d599d98a29a80/artifacts'


In [17]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1730972804685, experiment_id='1', last_update_time=1730972804685, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1730972599645, experiment_id='0', last_update_time=1730972599645, lifecycle_stage='active', name='Default', tags={}>]

## interacting with model registery

In [33]:
from mlflow.tracking import MlflowClient

client = MlflowClient("http://127.0.0.1:5000/")

In [34]:
client.search_registered_models()

[]

In [36]:
# Access run_id from the info attribute of the first run returned by search_runs
run_id = client.search_runs(1)[0].info.run_id

# Register the model with MLflow
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)



Successfully registered model 'iris-classifier'.
2024/11/07 09:55:53 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1730973353144, current_stage='None', description='', last_updated_timestamp=1730973353144, name='iris-classifier', run_id='5e8639781735427ba15d599d98a29a80', run_link='', source='mlflow-artifacts:/1/5e8639781735427ba15d599d98a29a80/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>

# Senario-3 : Multiple Data Scientist working on multiple modes

## setup
MLflow setup:
 * remote tracking server (ec2)
  * backend store = sqlite databse
  * artifacts store = s3 bucket

In [39]:
import mlflow
import os